In [1]:
from pyspark import SparkContext

import os
import sys

try:
    sc.stop()
except:
    pass
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

from pyspark import SparkConf
from pyspark import SparkContext

#HDFS_MASTER = 'hadoop-master'
conf = SparkConf()
conf.setMaster('yarn') # 'yarn' / 'local'
conf.setAppName('spark-test')
sc = SparkContext(conf=conf)

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/02 07:49:19 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/08/02 07:49:24 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!
23/08/02 07:49:24 WARN lineage.LineageWriter: Lineage directory /var/log/spark/lineage doesn't exist or is not writable. Lineage for this application will be disabled.


In [2]:
sqlContext

In [3]:
# Загрузка файла CSV из HDFS в RDD
rdd_csv = sc.textFile("hdfs:///user/std03/Salary_Data_Based_country_and_race.csv")

In [4]:
rdd_csv

hdfs:///user/std03/Salary_Data_Based_country_and_race.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

Задание 1. Подсчитать общее количество записей в CSV-файле.

In [5]:
rows = rdd_csv.count()
print(f"DataFrame Rows count : {rows}")

[Stage 0:=============================>                             (1 + 1) / 2]

DataFrame Rows count : 6705


In [6]:
rdd_csv.count()

6705

Задание 2. Найти все строки в CSV-файле, содержащие определенное ключевое слово.

In [7]:
rdd_csv.collect()[:][1]

"0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0,UK,White"

In [8]:
slovo = 'Data Engineer'

In [9]:
rdd_csv.filter(lambda x: x if slovo in x else False).collect()

['270,45.0,Male,PhD,Senior Data Engineer,16.0,150000.0,Australia,Asian',
 '303,45.0,Male,PhD,Senior Data Engineer,16.0,150000.0,Canada,Black',
 '344,44.0,Female,PhD,Senior Data Engineer,16.0,160000.0,Canada,Asian',
 '368,44.0,Female,PhD,Senior Data Engineer,16.0,160000.0,Australia,White']

Задание 3. Подсчитать сумму числовых значений в определенном столбце CSV-файла. 

In [10]:
# столбец 'Age'
r = rdd_csv.map(lambda x: x.split(',')[1]).collect()[1:]
r = filter(None,r)
# while '' in r:
#     r.remove('')
s = map(float,r)
sum(s)

225327.0

Задание 4. Получить случайную выборку определенного количества записей из CSV-файла.

In [11]:
rdd = rdd_csv.sample(False, 0.1, 42)
# rdd.collect()

In [12]:
rdd.count()

661

Задание 5. Посчитать количество уникальных слов в csv-файле.

In [13]:
rdd_csv.map(lambda line: line.split(',')).collect()[0:2]

[['',
  'Age',
  'Gender',
  'Education Level',
  'Job Title',
  'Years of Experience',
  'Salary',
  'Country',
  'Race'],
 ['0',
  '32.0',
  'Male',
  "Bachelor's",
  'Software Engineer',
  '5.0',
  '90000.0',
  'UK',
  'White']]

In [14]:
indx = [2, 3, 4, 7, 8]

In [15]:
total = 0
for i in indx:
    rdd = rdd_csv.map(lambda line: line.split(',')[i]).collect()[1:]
    unique = set(rdd) 
    j = len(unique)
    total = total + j

total

221

In [16]:
rdd_csv.map(lambda line: line.split(',')[2]).distinct().count()

5

Задание 6. Найти наиболее часто встречающееся значение в определенном столбце CSV-файла.

In [17]:
import collections
all_words = []
for i in indx:
    rdd = rdd_csv.map(lambda line: line.split(',')[i]).collect()[1:]
    for j in rdd:
            all_words.append(j)
counter = collections.Counter(all_words)
counter.most_common(10)

[('Male', 3674),
 ('Female', 3014),
 ("Bachelor's Degree", 2267),
 ('White', 1968),
 ('Asian', 1603),
 ("Master's Degree", 1573),
 ('PhD', 1368),
 ('USA', 1360),
 ('China', 1343),
 ('UK', 1337)]